In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [2]:
path = '/Users/sparajuli/Downloads/chromedriver'

In [4]:
def apply(job, location, name, email, phone):
    driver = webdriver.Chrome(path)
    #Link to indeed home
    driver.get("https://www.indeed.com/?from=gnav-jobsearch--jasx")
    jobtitle = driver.find_element_by_name("q")
    jobtitle.send_keys(job)
    where = driver.find_element_by_name("l")
    #Clears the default location on indeed
    #hacky solution - maxlen allowed for place is 62, so just press backspace 62 times lol
    for _ in range(62):
        where.send_keys(Keys.BACKSPACE);
    where.send_keys(location)
    where.send_keys(Keys.RETURN)
    
    ####Applying####
    cards = driver.find_elements_by_class_name("jobsearch-SerpJobCard")
#     for i in range(len(jobs)):
#         jobs[i].send_keys(Keys.RETURN)
    for card in cards:
        # only click the ones with indeed easy apply
        if "Easily apply" in card.get_attribute('innerHTML'):
            link = card.find_element_by_xpath(".//a[@class='jobtitle turnstileLink ']")
            actions = ActionChains(driver)
            actions.key_down(Keys.COMMAND).click(link).key_up(Keys.CONTROL).perform()
            driver.switch_to.window(driver.window_handles[-1])
            apply = driver.find_elements_by_tag_name("button")
            apply[2].click()
            break
    time.sleep(1)
#     html = driver.page_source
    iframe = driver.find_elements_by_tag_name("iframe")
    framenamesrc = [(i.get_attribute('name'), i.get_attribute('src')) for i in iframe]
    for framename, src in framenamesrc:
        if framename.startswith('indeedapply'):
            driver.switch_to.frame(framename)
            #Nested iframe
            iframeinner = driver.find_elements_by_tag_name("iframe")
            srcinner = [i.get_attribute('src') for i in iframeinner]
            for innersrc in srcinner:
                #Trouble working with iframe itself, so need to find inner link
                if innersrc.startswith('https://apply.indeed.com/indeedapply/'):
                    print(innersrc)
                    driver.get(innersrc)
                    break
            break
    namefield = driver.find_element_by_name("applicant.name")
    #Need to send random string first for some reason
    namefield.send_keys("")
    namefield.send_keys(name)
    emailfield = driver.find_element_by_name("applicant.email")
    emailfield.send_keys(email)
    phonefield = driver.find_element_by_name("applicant.phoneNumber")
    phonefield.send_keys(phone)
    time.sleep(5)
    driver.quit()

In [5]:
apply("Driver", "East Brunswick, NJ", "First Last", "first@last.com", "9494949494")

https://apply.indeed.com/indeedapply/resumeapply?jobUrl=https%3A%2F%2Fwww.indeed.com%2Fviewjob%3Fjk%3Dbb2ebfe8899dcb61&postUrl=https%3A%2F%2Fdradisindeedapply.sandbox.indeed.net%2Fprocess-indeedapply&continueUrl=http%3A%2F%2Fwww.indeed.com%2Fviewjob%3Fcmp%3DSDB-Package-%2526-Delivery%26t%3DFedEx%2BGround%2BDeliver%2BDriver%26jk%3Dbb2ebfe8899dcb61%26sjdu%3DQwrRXKrqZ3CNX5W-O9jEvTW-fRtKOrfG_wLZ1jc7avH4YjLRZc6ogoYVF4meJTftTY6zyQ9wBJxvk2zvmmphOyStQbJfNvovzxwwL3FUnNU%26tk%3D1eamilkii2uu1000%26adid%3D345749143%26pub%3D4a1b367933fd867b19b072952f68dceb%26vjs%3D3%26applied%3D1%26sita%3D1%26from%3Dpostapply&questions=nigma%3A%2F%2FWmh2b11WEFRdFUZfREJq%3Flocale%3Den_US%26v%3D3&phone=optional&coverletter=OPTIONAL&jobId=a17e17f3015c9b57ed54&jk=bb2ebfe8899dcb61&jobTitle=FedEx%20Ground%20Deliver%20Driver%20(NON%20CDL%20)&jobCompany=SDB%20Package%20%26%20Delivery&jobLocation=Hamilton%20Township%2C%20NJ&apiToken=aa102235a5ccb18bd3668c0e14aa3ea7e2503cfac2a7a9bf3d6549899e125af4&advNum=1152931073936474&pin

In [ ]:
#     try:
#         apply_now = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "indeed-apply-button"))
#     )
#     except:
#         apply_now = None
#         driver.quit()